In [21]:
# Import numpy and OpenCV
import numpy as np
import cv2
 
# Read input video
cap = cv2.VideoCapture('org_octo.mp4')
 
# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
 
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
 
n_frames , w , h 

(100, 960, 540)

In [22]:
import matplotlib.pyplot as plt

In [23]:
# Read first frame
_, prev = cap.read() 
 
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

# plt.imshow(prev_gray)

In [24]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 
 
for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)
 
  # Read next frame
  success, curr = cap.read()
  if not success:
    break
 
  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 
 
  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
 
  # Sanity check
  assert prev_pts.shape == curr_pts.shape 
 
  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]
 
  #Find transformation matrix
  m = cv2.estimateAffine2D(prev_pts, curr_pts) #will only work with OpenCV-3 or less
 
  # Extract traslation
#   print("fff" , m)
  dx = m[0][0,2]
  dy = m[0][1,2]
 
  # Extract rotation angle
  da = np.arctan2(m[0][1,0], m[0][0,0])
 
  # Store transformation
  transforms[i] = [dx,dy,da]
 
  # Move to next frame
  prev_gray = curr_gray
 
  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))


Frame: 0/100 -  Tracked points : 191
Frame: 1/100 -  Tracked points : 186
Frame: 2/100 -  Tracked points : 184
Frame: 3/100 -  Tracked points : 186
Frame: 4/100 -  Tracked points : 191
Frame: 5/100 -  Tracked points : 184
Frame: 6/100 -  Tracked points : 186
Frame: 7/100 -  Tracked points : 173
Frame: 8/100 -  Tracked points : 185
Frame: 9/100 -  Tracked points : 195
Frame: 10/100 -  Tracked points : 187
Frame: 11/100 -  Tracked points : 186
Frame: 12/100 -  Tracked points : 193
Frame: 13/100 -  Tracked points : 198
Frame: 14/100 -  Tracked points : 191
Frame: 15/100 -  Tracked points : 183
Frame: 16/100 -  Tracked points : 184
Frame: 17/100 -  Tracked points : 184
Frame: 18/100 -  Tracked points : 187
Frame: 19/100 -  Tracked points : 192
Frame: 20/100 -  Tracked points : 196
Frame: 21/100 -  Tracked points : 189
Frame: 22/100 -  Tracked points : 192
Frame: 23/100 -  Tracked points : 192
Frame: 24/100 -  Tracked points : 191
Frame: 25/100 -  Tracked points : 197
Frame: 26/100 -  Track

In [25]:
trajectory = np.cumsum(transforms, axis=0)

In [26]:
SMOOTHING_RADIUS = 5

def movingAverage(curve, radius):
  window_size = 2 * radius + 1
  # Define the filter
  f = np.ones(window_size)/window_size
  # Add padding to the boundaries
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge')
  # Apply convolution
  curve_smoothed = np.convolve(curve_pad, f, mode='same')
  # Remove padding
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed


def smooth(trajectory):
  smoothed_trajectory = np.copy(trajectory)
  # Filter the x, y and angle curves
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)
 
  return smoothed_trajectory


trajectory = np.cumsum(transforms, axis=0)

smoothed_trajectory = smooth(trajectory)
difference = smoothed_trajectory - trajectory
 
# Calculate newer transformation array
transforms_smooth = transforms + difference

In [27]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame

In [28]:
# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')

fps = 3
# Set up output video
out = cv2.VideoWriter('video_out.mp4', fourcc, fps, (w, h))

In [29]:
# Reset stream to first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 

all_frames = []

cc = 0 
# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  success, frame = cap.read()
  if not success:
    break
 
  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]
 
  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy
 
  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))
 
  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 
 
  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])
 
  print(cc , " : ", frame_out.shape)
  cc+=1
  all_frames.append(frame_out)
  cv2.imshow("Before and After", frame_out)

  
  cv2.waitKey(10)
  out.write(frame_out)

0  :  (540, 1920, 3)
1  :  (540, 1920, 3)
2  :  (540, 1920, 3)
3  :  (540, 1920, 3)
4  :  (540, 1920, 3)
5  :  (540, 1920, 3)
6  :  (540, 1920, 3)
7  :  (540, 1920, 3)
8  :  (540, 1920, 3)
9  :  (540, 1920, 3)
10  :  (540, 1920, 3)
11  :  (540, 1920, 3)
12  :  (540, 1920, 3)
13  :  (540, 1920, 3)
14  :  (540, 1920, 3)
15  :  (540, 1920, 3)
16  :  (540, 1920, 3)
17  :  (540, 1920, 3)
18  :  (540, 1920, 3)
19  :  (540, 1920, 3)
20  :  (540, 1920, 3)
21  :  (540, 1920, 3)
22  :  (540, 1920, 3)
23  :  (540, 1920, 3)
24  :  (540, 1920, 3)
25  :  (540, 1920, 3)
26  :  (540, 1920, 3)
27  :  (540, 1920, 3)
28  :  (540, 1920, 3)
29  :  (540, 1920, 3)
30  :  (540, 1920, 3)
31  :  (540, 1920, 3)
32  :  (540, 1920, 3)
33  :  (540, 1920, 3)
34  :  (540, 1920, 3)
35  :  (540, 1920, 3)
36  :  (540, 1920, 3)
37  :  (540, 1920, 3)
38  :  (540, 1920, 3)
39  :  (540, 1920, 3)
40  :  (540, 1920, 3)
41  :  (540, 1920, 3)
42  :  (540, 1920, 3)
43  :  (540, 1920, 3)
44  :  (540, 1920, 3)
45  :  (540, 1920, 3

In [30]:
import cv2
import os


def convert_frames_to_video(frames1, output_path, fps):
    
    # Get frame dimensions from the first frame
    frame = frames1[0]
    height, width, channels = frame.shape
    
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for frame in frames1:
        # Read the frame image
        
        # Write the frame to the video
        video.write(frame)
    
    # Release the video writer
    video.release()


output_folder = " "
output_video_path = "path_to_output_video.mp4"
fps = 5

# Extract frames from the video

# Convert frames back to video
convert_frames_to_video(all_frames, output_video_path, fps)

: 